In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import KFold


In [45]:
red_q = pd.read_csv('winequality-red.csv', sep=';')
red_q_tv = red_q['quality']
red_q = red_q.drop('quality', axis=1)
white_q = pd.read_csv('winequality-white.csv', sep=';')
white_q_tv = white_q['quality']
white_q = white_q.drop('quality', axis=1)

#  red_q = red_q.
max_depth = 5
kf =  KFold(n_splits=5, shuffle=True)
scores_total = 0
for i , (train_index, test_index) in enumerate(kf.split(red_q)):
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = red_q.iloc[train_index], red_q.iloc[test_index]
    y_train, y_test = red_q_tv.iloc[train_index], red_q_tv.iloc[test_index]
    regr_1 = DecisionTreeRegressor(max_depth=max_depth)
    tree = regr_1.fit(X_train, y_train)
    score  = tree.score(X_test, y_test)
    scores_total += score
    print(f'The score for fold {i} is', score)
    print('-------------------')

print(f'average score for a max_depth of {max_depth} is {scores_total / kf.get_n_splits()}')

dtr = DecisionTreeRegressor(max_depth=max_depth)
tree_red = dtr.fit(red_q, red_q_tv)

dtr2= DecisionTreeRegressor(max_depth=max_depth)
tree_white = dtr2.fit(white_q, white_q_tv)
print('Red tree score is', tree_red.score(red_q, red_q_tv))
print('White tree score is', tree_white.score(white_q, white_q_tv))

# importances = tree_red.feature_importances_
# print(importances)



The score for fold 0 is 0.27868090297435644
-------------------
The score for fold 1 is 0.21325334094878645
-------------------
The score for fold 2 is 0.28928317342967214
-------------------
The score for fold 3 is 0.30921813955339805
-------------------
The score for fold 4 is 0.3071967998682499
-------------------
average score for a max_depth of 5 is 0.27952647135489256
Red tree score is 0.4627584023894983
White tree score is 0.3666891190470236


In [57]:
red_p = pd.read_csv('Red.csv', sep=',')
red_p['Rating'] = red_p['Rating'] * 2
print(tree_red.predict(red_q.iloc[0:2]))
print(red_p[['Rating', 'Price']])


[5.10576923 5.29577465]
      Rating  Price
0        8.4  95.00
1        8.6  15.50
2        7.8   7.45
3        7.0   8.72
4        7.8  29.15
...      ...    ...
8661     7.6  16.47
8662     8.0  20.09
8663     7.4  23.95
8664     7.0   6.21
8665     6.8   7.88

[8666 rows x 2 columns]
